In [1]:
%load_ext autotime

import os, math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm, trange

In [2]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Input, BatchNormalization, Bidirectional
from sklearn.metrics import mean_squared_error, mean_absolute_error

def model(test_data, train_data):
    test_time = test_data['time']
    test_gl_value = test_data['gl_value']
    
    test_data.drop(columns = ['time'], inplace = True)
    train_data.rename(columns = {'bolus_type_normal dual':'bolus_type_normal_dual', 'bolus_type_square dual':'bolus_type_square_dual'}, inplace = True)
    train_data.drop(columns = ['time'], inplace = True)
    
    empty_train_col = [0]*len(train_data)
    for i, item in enumerate(test_data.columns):
        if item not in train_data.columns:
            train_data.insert(i, item, empty_train_col)

    empty_test_col = [0]*len(test_data)
    for i, item in enumerate(train_data.columns):
        if item not in test_data.columns:
            test_data.insert(i, item, empty_test_col)
    
    X_data = train_data.drop(columns = ['gl_predict'])
    y_data = train_data[['gl_predict']]
    input_dim = X_data.shape[1]
    
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()
    scaler_x.fit(X_data)
    scaler_y.fit(y_data)
    scaled_X_train_data = scaler_x.transform(X_data)
    scaled_y_train_data = scaler_y.transform(y_data)
    scaled_X_train_data = np.reshape(scaled_X_train_data, (scaled_X_train_data.shape[0], 1, scaled_X_train_data.shape[1]))
    
    model = Sequential()
    model.add(Bidirectional(LSTM(128, input_shape = (scaled_X_train_data.shape[1], scaled_X_train_data.shape[2]))))
    model.add(Dense(150, activation = 'relu'))
    model.add(Dropout(0.20))
    model.add(Dense(100, activation = 'relu'))
    model.add(Dropout(0.15))
    model.add(Dense(50, activation = 'relu'))
    model.add(Dense(20, activation = 'relu'))
    model.add(Dense(1, activation = 'relu'))
    model.compile(loss = 'mse', optimizer = 'adam')
    model.fit(scaled_X_train_data, scaled_y_train_data, epochs = 200, batch_size = 32, shuffle = False)
    
    X_test_data = test_data.drop(columns = ['gl_predict'])
    y_test_data = test_data[['gl_predict']]
    scaled_X_test_data = scaler_x.transform(X_test_data)
    scaled_X_test_data = np.reshape(scaled_X_test_data, (scaled_X_test_data.shape[0], 1, scaled_X_test_data.shape[1]))
    prediction = model.predict(scaled_X_test_data, batch_size = 32)
    scaled_prediction = scaler_y.inverse_transform(prediction)
    
    final_output = pd.DataFrame(columns = ['timestamp', 'predicted_BGL_value'])
    final_output['timestamp'] = test_time
#     final_output['true_BGL_value'] = test_gl_value
    final_output['predicted_BGL_value'] = scaled_prediction
    
    return (final_output, math.sqrt(mean_squared_error(scaled_prediction, y_test_data)), mean_absolute_error(scaled_prediction, y_test_data))

Using TensorFlow backend.


time: 2.57 s


In [3]:
test_data_list = ['540-ws-training.csv', '544-ws-training.csv', '552-ws-training.csv', '567-ws-training.csv', '584-ws-training.csv', '596-ws-training.csv']

rmse_list = {}
mae_list = {}
for file in os.listdir('../data/derived/5_min_complete_dataset/'):
    if file in test_data_list:
        train_data = pd.read_csv('../data/derived/5_min_complete_dataset/'+file)
        test_data = pd.read_csv('../data/derived_test/5_min_complete_dataset/'+file[0:7]+'testing.csv')
        final_output, rmse_error, mae_error = model(test_data, train_data)
        rmse_list[file] = rmse_error
        mae_list[file] = mae_error
        print ('{} - {}'.format(file, rmse_error))
        print ('{} - {}'.format(file, mae_error))
        final_output.to_csv('../data/output/bidirectional_lstm/'+file[0:7]+'result_5min.csv')

Epoch 1/200
13247/13247 [==============================] - 5s 401us/step - loss: 0.0123
Epoch 2/200
13247/13247 [==============================] - 3s 258us/step - loss: 0.0030
Epoch 3/200
13247/13247 [==============================] - 3s 245us/step - loss: 0.0016
Epoch 4/200
13247/13247 [==============================] - 3s 240us/step - loss: 0.0022
Epoch 5/200
13247/13247 [==============================] - 3s 250us/step - loss: 0.0022
Epoch 6/200
13247/13247 [==============================] - 3s 227us/step - loss: 0.0015
Epoch 7/200
13247/13247 [==============================] - 3s 248us/step - loss: 0.0017
Epoch 8/200
13247/13247 [==============================] - 3s 244us/step - loss: 0.0028
Epoch 9/200
13247/13247 [==============================] - 3s 229us/step - loss: 0.0012
Epoch 10/200
13247/13247 [==============================] - 3s 239us/step - loss: 0.0013
Epoch 11/200
13247/13247 [==============================] - 3s 246us/step - loss: 0.0013
Epoch 12/200
13247/13247 [====

13247/13247 [==============================] - 5s 400us/step - loss: 7.7738e-04
Epoch 91/200
13247/13247 [==============================] - 5s 352us/step - loss: 7.8511e-04
Epoch 92/200
13247/13247 [==============================] - 4s 339us/step - loss: 7.4243e-04
Epoch 93/200
13247/13247 [==============================] - 5s 353us/step - loss: 7.6218e-04
Epoch 94/200
13247/13247 [==============================] - 5s 352us/step - loss: 7.7873e-04
Epoch 95/200
13247/13247 [==============================] - 4s 337us/step - loss: 7.1547e-04
Epoch 96/200
13247/13247 [==============================] - 4s 330us/step - loss: 7.1429e-04
Epoch 97/200
13247/13247 [==============================] - 5s 342us/step - loss: 7.3146e-04
Epoch 98/200
13247/13247 [==============================] - 4s 317us/step - loss: 7.1633e-04
Epoch 99/200
13247/13247 [==============================] - 4s 315us/step - loss: 7.5062e-04
Epoch 100/200
13247/13247 [==============================] - 5s 349us/step - loss: 

13247/13247 [==============================] - 3s 250us/step - loss: 6.8379e-04
Epoch 178/200
13247/13247 [==============================] - 3s 261us/step - loss: 6.5823e-04
Epoch 179/200
13247/13247 [==============================] - 3s 245us/step - loss: 6.5747e-04
Epoch 180/200
13247/13247 [==============================] - 4s 267us/step - loss: 6.6631e-04
Epoch 181/200
13247/13247 [==============================] - 3s 257us/step - loss: 6.6714e-04
Epoch 182/200
13247/13247 [==============================] - 3s 259us/step - loss: 6.4640e-04
Epoch 183/200
13247/13247 [==============================] - 3s 251us/step - loss: 7.1409e-04
Epoch 184/200
13247/13247 [==============================] - 4s 267us/step - loss: 6.6921e-04
Epoch 185/200
13247/13247 [==============================] - 3s 264us/step - loss: 6.5482e-04
Epoch 186/200
13247/13247 [==============================] - 4s 275us/step - loss: 6.7050e-04
Epoch 187/200
13247/13247 [==============================] - 3s 241us/step

13535/13535 [==============================] - 3s 207us/step - loss: 5.6008e-04
Epoch 65/200
13535/13535 [==============================] - 3s 224us/step - loss: 5.7556e-04
Epoch 66/200
13535/13535 [==============================] - 3s 207us/step - loss: 6.5612e-04
Epoch 67/200
13535/13535 [==============================] - 3s 211us/step - loss: 5.6070e-04
Epoch 68/200
13535/13535 [==============================] - 3s 223us/step - loss: 5.1588e-04
Epoch 69/200
13535/13535 [==============================] - 3s 212us/step - loss: 6.3315e-04
Epoch 70/200
13535/13535 [==============================] - 3s 212us/step - loss: 6.0115e-04
Epoch 71/200
13535/13535 [==============================] - 3s 191us/step - loss: 5.1219e-04
Epoch 72/200
13535/13535 [==============================] - 3s 211us/step - loss: 5.3049e-04
Epoch 73/200
13535/13535 [==============================] - 3s 214us/step - loss: 5.3964e-04
Epoch 74/200
13535/13535 [==============================] - 3s 209us/step - loss: 5

13535/13535 [==============================] - 2s 167us/step - loss: 5.5549e-04
Epoch 152/200
13535/13535 [==============================] - 2s 176us/step - loss: 5.1907e-04
Epoch 153/200
13535/13535 [==============================] - 2s 172us/step - loss: 4.8901e-04
Epoch 154/200
13535/13535 [==============================] - 2s 175us/step - loss: 4.6527e-04
Epoch 155/200
13535/13535 [==============================] - 2s 173us/step - loss: 4.9071e-04
Epoch 156/200
13535/13535 [==============================] - 2s 180us/step - loss: 4.5359e-04
Epoch 157/200
13535/13535 [==============================] - 2s 172us/step - loss: 4.7945e-04
Epoch 158/200
13535/13535 [==============================] - 2s 168us/step - loss: 5.1402e-04
Epoch 159/200
13535/13535 [==============================] - 3s 185us/step - loss: 5.7377e-04
Epoch 160/200
13535/13535 [==============================] - 2s 179us/step - loss: 4.7169e-04
Epoch 161/200
13535/13535 [==============================] - 2s 169us/step

...

11096/11096 [==============================] - 2s 155us/step - loss: 5.5696e-04
Epoch 189/200
11096/11096 [==============================] - 2s 154us/step - loss: 5.2400e-04
Epoch 190/200
11096/11096 [==============================] - 2s 156us/step - loss: 5.3150e-04
Epoch 191/200
11096/11096 [==============================] - 2s 149us/step - loss: 5.1133e-04
Epoch 192/200
11096/11096 [==============================] - 2s 149us/step - loss: 5.6833e-04
Epoch 193/200
11096/11096 [==============================] - 2s 148us/step - loss: 5.1477e-04
Epoch 194/200
11096/11096 [==============================] - 2s 148us/step - loss: 5.0600e-04
Epoch 195/200
11096/11096 [==============================] - 2s 150us/step - loss: 5.1465e-04
Epoch 196/200
11096/11096 [==============================] - 2s 158us/step - loss: 5.0760e-04
Epoch 197/200
11096/11096 [==============================] - 2s 157us/step - loss: 5.9270e-04
Epoch 198/200
11096/11096 [==============================] - 2s 151us/step

13104/13104 [==============================] - 2s 154us/step - loss: 3.2019e-04
Epoch 76/200
13104/13104 [==============================] - 2s 160us/step - loss: 2.8768e-04
Epoch 77/200
13104/13104 [==============================] - 2s 159us/step - loss: 3.1652e-04
Epoch 78/200
13104/13104 [==============================] - 2s 145us/step - loss: 2.8405e-04
Epoch 79/200
13104/13104 [==============================] - 2s 141us/step - loss: 2.4671e-04
Epoch 80/200
13104/13104 [==============================] - 2s 146us/step - loss: 2.8255e-04
Epoch 81/200
13104/13104 [==============================] - 2s 159us/step - loss: 3.5713e-04
Epoch 82/200
13104/13104 [==============================] - 2s 155us/step - loss: 2.9604e-04
Epoch 83/200
13104/13104 [==============================] - 2s 152us/step - loss: 2.7526e-04
Epoch 84/200
13104/13104 [==============================] - 2s 146us/step - loss: 2.9865e-04
Epoch 85/200
13104/13104 [==============================] - 2s 148us/step - loss: 2

13104/13104 [==============================] - 1s 113us/step - loss: 3.1590e-04
Epoch 163/200
13104/13104 [==============================] - 1s 113us/step - loss: 2.3886e-04
Epoch 164/200
13104/13104 [==============================] - 2s 115us/step - loss: 2.6394e-04
Epoch 165/200
13104/13104 [==============================] - 1s 114us/step - loss: 2.3014e-04
Epoch 166/200
13104/13104 [==============================] - 1s 112us/step - loss: 2.4841e-04
Epoch 167/200
13104/13104 [==============================] - 1s 111us/step - loss: 2.2880e-04
Epoch 168/200
13104/13104 [==============================] - 2s 116us/step - loss: 2.4165e-04
Epoch 169/200
13104/13104 [==============================] - 2s 117us/step - loss: 2.2719e-04
Epoch 170/200
13104/13104 [==============================] - 2s 123us/step - loss: 2.3175e-04
Epoch 171/200
13104/13104 [==============================] - 2s 129us/step - loss: 2.2311e-04
Epoch 172/200
13104/13104 [==============================] - 2s 126us/step

13104/13104 [==============================] - 2s 117us/step - loss: 5.1310e-04
Epoch 50/200
13104/13104 [==============================] - 1s 111us/step - loss: 4.7638e-04
Epoch 51/200
13104/13104 [==============================] - 1s 112us/step - loss: 5.7706e-04
Epoch 52/200
13104/13104 [==============================] - 1s 110us/step - loss: 4.6248e-04
Epoch 53/200
13104/13104 [==============================] - 1s 114us/step - loss: 5.5242e-04
Epoch 54/200
13104/13104 [==============================] - 2s 115us/step - loss: 4.4166e-04
Epoch 55/200
13104/13104 [==============================] - 2s 115us/step - loss: 4.6648e-04
Epoch 56/200
13104/13104 [==============================] - 1s 114us/step - loss: 4.4288e-04
Epoch 57/200
13104/13104 [==============================] - 2s 123us/step - loss: 5.2173e-04
Epoch 58/200
13104/13104 [==============================] - 1s 114us/step - loss: 4.4553e-04
Epoch 59/200
13104/13104 [==============================] - 1s 112us/step - loss: 4

13104/13104 [==============================] - 1s 95us/step - loss: 4.4666e-04
Epoch 138/200
13104/13104 [==============================] - 1s 95us/step - loss: 4.0669e-04
Epoch 139/200
13104/13104 [==============================] - 1s 94us/step - loss: 4.2929e-04
Epoch 140/200
13104/13104 [==============================] - 1s 95us/step - loss: 4.1327e-04
Epoch 141/200
13104/13104 [==============================] - 1s 97us/step - loss: 4.3248e-04
Epoch 142/200
13104/13104 [==============================] - 1s 96us/step - loss: 4.2369e-04
Epoch 143/200
13104/13104 [==============================] - 1s 97us/step - loss: 4.2104e-04
Epoch 144/200
13104/13104 [==============================] - 1s 96us/step - loss: 4.1254e-04
Epoch 145/200
13104/13104 [==============================] - 1s 95us/step - loss: 4.1673e-04
Epoch 146/200
13104/13104 [==============================] - 1s 98us/step - loss: 4.1992e-04
Epoch 147/200
13104/13104 [==============================] - 1s 96us/step - loss: 4.

In [4]:
rmse_list

{'584-ws-training.csv': 15.556218127851615,
 '567-ws-training.csv': 11.21187252350762,
 '596-ws-training.csv': 11.86922963134945,
 '552-ws-training.csv': 12.446494239238735,
 '544-ws-training.csv': 10.698685256780994,
 '540-ws-training.csv': 15.183328561491516}

time: 3.01 ms


In [5]:
mae_list

{'584-ws-training.csv': 13.250312886719076,
 '567-ws-training.csv': 8.57838705109387,
 '596-ws-training.csv': 8.853997789010615,
 '552-ws-training.csv': 11.228864618119303,
 '544-ws-training.csv': 7.6121815439616665,
 '540-ws-training.csv': 13.132645723675825}

time: 7.78 ms


In [6]:
from statistics import pstdev, mean

rmse_values = rmse_list.values()
mae_values = mae_list.values()

print ('Mean - RMSE : {}'.format(mean(rmse_values)))
print ('Standard Deviation - RMSE : {}'.format(pstdev(rmse_values)))

print ('Mean - MAE : {}'.format(mean(mae_values)))
print ('Standard Deviation - MAE : {}'.format(pstdev(mae_values)))

Mean - RMSE : 12.827638056703321
Standard Deviation - RMSE : 1.879777582630365
Mean - MAE : 10.442731602096726
Standard Deviation - MAE : 2.2266576273052046
time: 6.93 ms
